In [39]:
%load_ext autoreload
%autoreload 2


import queries.min_date_nbh_query as MinDateNBHQuery
import queries.nbh_trend_query as NBHTrendQuery
import queries.nbh_trend_date_query as NBHTrendDateQuery

import executors.bigquery_executor as BigQueryExecutor
import helpers
import importlib

importlib.reload(MinDateNBHQuery)
importlib.reload(NBHTrendQuery)
importlib.reload(NBHTrendDateQuery)

importlib.reload(BigQueryExecutor)
importlib.reload(helpers)

import concurrent.futures
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Context
# Some reports were raised about the results returned from google engine for some queries as halussinations, 
# the issue is that we don't have a way to detect those halussinations, the only characteristic we know is that the number of hits (nbh) returned from google is 400 for those halussinations.

# Issues
# Its not guranteed that 400 is always a halussination, it could be a valid result, but we don't have a way to detect that.
# It's not known if there are other queries that are affected by this issue, and we don't have a way to detect those halussinations.

# Approach
# we have a list of queries that are known to be halussinations which were flaged to google in the Google 400 products cases Attribution CSV report,
# we also have historicly data for the queries in the even_misc, the only metric that might give us something to work with is the nbh (number of hits) returned from google,
# we will assume that if the nbh is 400 for a query, then it is a halussination, as this is the only characteristic we know about the halussinations,
# we can then detect the first time a halussination was detected for a query and we will assume that this is the start of the halussination,
# then we will consider the historcial data prior to the start of the halussination as the baseline of the analysis over the nbh,
# we will then compare the frequency of nbh before and after the start of the halussination, to get some insights about the halussination,
# along with some statitical metrics like mean and std deviation, to get some insights about the halussination,
# hopefully this will give us some insights on how to detect a halussination and to detect other queries that are affected by this issue.

In [ ]:
# 1st step is to get the list of queries that are known to be halussinations, and the start date of the halussination using the first time the nbh was 400.

In [31]:
# Get english search terms which are flagged to google as halussinations (ignore arabic as they causes some issues in the encoding)

csv_path = "./Google 400 products cases Attribution Tokens - Sheet1.csv"
search_terms_df = pd.read_csv(csv_path)
search_terms = search_terms_df['search_query'].tolist()

# Filter out Arabic search terms
search_terms = [term for term in search_terms if not helpers.is_arabic(term)]

# Remove duplicates
search_terms = list(set(search_terms))
search_terms.append("shoes")
print(f"Number of search terms: {len(search_terms)} , search terms: {search_terms}")

Number of search terms: 611 , search terms: ['tshirt women', 'light', 'brand Tshirt', 'pants for men', 'o r shoes', 'steel wardrobe', 'grill sandwich maker', 'bags for ladies', 'men makeup', 'womens shoe', 'sport shoes', 'maxi mini dress', 'tinsel', 'ladies footwear', 'shoe mens', 'photocard holdet', 'samsung galaxy a52 5g screen protector', 'anti slip shoes', 'oversize t sorts', 'dress', 'tshirts for ladies', 'x shus', 'colloidal silver', 'sport shoes for men', 'polo t shirt', 'king size fitted bed sheet', 'women tshirt', 'new phone', 'mobile 200', 'shouse for man', 'brown table mat', 'hoodi', 'dress vermoda', 'pants', 'samsung a25', 'gaming birthday decoration', 'ladies bag', 'slip ONS', 'costum', 'tee', 'shoes for mens', 'women dress evening', 'rubning shoes', 'mens fashion shirt', 'angle grinder pad', 'wifi adapter for pc', 'on on shoes', 'polo tee', 's24 ultra screen protector', 'footwear for men', 'type c chargee', 'night suit ladies', 'lattafa perfume for men', 's23 ultra lens p

In [36]:
project_id = "noonbisearch"
duration_months = 12 # Number of months to look back for the nbh 400 trend
target_nbh = 400

def get_query_start_halussination(search_term):
    """
    Get the start date of the halussination for a given search term
    """
    try:
        min_date_query = MinDateNBHQuery.MinDateNBHQuery(search_term= search_term,project_id= project_id, duration_months=duration_months , target_nbh=target_nbh)
        result = BigQueryExecutor.BigQueryExecutor().execute_query(min_date_query,cache=True)

        if result is None:
            return {"search_query"  : search_term, "min_date": None}
        if result.empty:
            return {"search_query"  : search_term, "min_date": None}
        return {"search_query"  : search_term, "min_date": result['event_date'].values[0]}
    except Exception as e:
        return {"search_query"  : search_term, "min_date": None}

results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_term = {executor.submit(get_query_start_halussination, term): term for term in search_terms}
    for future in concurrent.futures.as_completed(future_to_term):
        result = future.result()
        results.append(result)


# Filter out queries that have no 400
results_without_400 = [result for result in results if result['min_date'] is None]

# Filter out queries that have 400 detected
results_with_400 = [result for result in results if result['min_date'] is not None]

# sort the results by min_date
results_with_400.sort(key=lambda x: x['min_date'])

# print the results
print(f"Results without 400 len {len(results_without_400)}:\n {results_without_400}\n")

print(f"Results with 400 len {len(results_with_400)}:\n {results_with_400}\n")
for result in results_with_400:
    print(f"Search term: {result['search_query']}, Min date: {result['min_date']}")

# store the 400 in a csv file
results_with_400_df = pd.DataFrame(results_with_400)
results_with_400_df.to_csv("./final_outputs/results_with_400_first_date.csv", index=False)

    

Query failed: Command '['bq', 'query', '--project_id=noonbisearch', '--format=json', '--nouse_legacy_sql', '\n            SELECT\n                DATE(event_time) AS event_date,\n                MIN(CAST(JSON_EXTRACT_SCALAR(event_misc, "$.nbh") AS BIGNUMERIC)) AS min_nbh\n            FROM\n                `noonprd-mp-analytics.noon_analytics_tool.raw_events_v2`\n            WHERE\n                event_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH) AND CURRENT_DATE()\n                AND property_code = \'noon\'\n                AND JSON_EXTRACT_SCALAR(event_misc, "$.mc") = "noon"\n                AND event_type = \'page_catalog\'\n                AND lower(JSON_EXTRACT_SCALAR(event_misc, "$.st")) = \'men\'s shoe\'\n                AND lower(JSON_EXTRACT_SCALAR(event_misc, "$.td.eid")) = \'google\'\n                AND JSON_VALUE(event_misc, "$.ed.cd.hit") = "false"\n                AND JSON_EXTRACT_SCALAR(event_misc, "$.st") IS NOT NULL\n                AND JSON_EXTRACT_SCAL

In [ ]:
# 2nd step is to get the nbh trend for the queries that have 400 detected (still not finished)